In [1]:
# This notebook will clean web statistics data by:
#
# 1) read a csv file containing web statistics from a hemispheres.com web site for an art gallery company
# 2) identify missing data
# 3) access a third party API to supply missing location data
# 4) update the web statistics data with the API's location data

In [2]:
import pandas as pd
import numpy as np

In [3]:
web_stats_df = pd.read_csv("statz_CSV_titles.csv")

In [4]:
np.random.seed(0)

In [5]:
# Gather some information about the web stats file and print some sample rows from it to better understand the file contents

In [6]:
web_stats_df.info()
web_stats_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333351 entries, 0 to 333350
Data columns (total 24 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   RECORDID    333351 non-null  int64  
 1   EPOCH       333351 non-null  int64  
 2   DATE        333351 non-null  int64  
 3   DATESTAMP   333351 non-null  int64  
 4   SESSION     333351 non-null  int64  
 5   IP          333351 non-null  object 
 6   HOSTNAME    333351 non-null  object 
 7   PAGE        333351 non-null  object 
 8   L1          237079 non-null  object 
 9   L2          90124 non-null   object 
 10  ARTID       333351 non-null  int64  
 11  EDITKEY     31629 non-null   object 
 12  FIRSTNAME   25206 non-null   object 
 13  LASTNAME    31565 non-null   object 
 14  CITY        129710 non-null  object 
 15  REGION      160898 non-null  object 
 16  REGIONCODE  160860 non-null  object 
 17  COUNTRY     331748 non-null  object 
 18  REF         333351 non-null  object 
 19  IP

,RECORDID,EPOCH,DATE,DATESTAMP,SESSION,IP,HOSTNAME,PAGE,L1,L2,...,CITY,REGION,REGIONCODE,COUNTRY,REF,IPOLD,BROWSER,MEMID,TYPE,PUBLISH
116201,467721,1625720700,7,20210707,75118331320707,75.118.33.132,d118-75-132-33.try.wideopenwest.com,Artist Gallery,Dorian Vallejo,26,...,Naperville,Illinois,IL,US,https://www.thbrennenfineart.com/artists.php,NaN,NaN,0,artist,yes
211718,563238,1639366651,12,20211212,154542492071212,154.54.249.207,crawl-prodd4-15.babbar.eu,Lew Brennan,Appapolo,NaN,...,NaN,NaN,NaN,US,unknown,NaN,NaN,0,artwork,yes
85498,437018,1621657260,21,20210521,681842122050521,68.184.212.205,068-184-212-205.res.spectrum.com,Artist Gallery,Wesley Wofford,48,...,Belleville,Illinois,IL,US,https://www.thbrennenfineart.com/artists.php,NaN,NaN,0,artist,yes
196564,548084,1636991535,15,20211115,1419881231115,141.98.81.23,141.98.81.23,Home Page,NaN,NaN,...,NaN,NaN,NaN,PA,unknown,NaN,NaN,0,NaN,yes
332743,733139,1661349620,24,20220824,6624975380824,66.249.75.38,crawl-66-249-75-38.googlebot.com,Giner Bueno,La Manana,NaN,...,NaN,NaN,NaN,US,unknown,NaN,NaN,0,artwork,yes


In [7]:
# calculate and print a count of all missing values. This will serve as control counts to compare with our work below

In [8]:
missing_values_count = web_stats_df.isnull().sum()
missing_values_count[0:24]

RECORDID           0
EPOCH              0
DATE               0
DATESTAMP          0
SESSION            0
IP                 0
HOSTNAME           0
PAGE               0
L1             96272
L2            243227
ARTID              0
EDITKEY       301722
FIRSTNAME     308145
LASTNAME      301786
CITY          203641
REGION        172453
REGIONCODE    172491
COUNTRY         1603
REF                0
IPOLD         333351
BROWSER       333351
MEMID              0
TYPE           96249
PUBLISH            0
dtype: int64

In [9]:
#Calculate total cells of the web status data frame, sum up the missing values and report the percentage

In [10]:
web_stats_total_cells = np.product(web_stats_df.shape)
web_stats_total_missing_values = missing_values_count.sum()
(web_stats_total_missing_values/web_stats_total_cells) * 100

32.051938747246396

In [11]:
# Begin the process of filling in some of the most important missing data using data returned from an API call 
# to the IP-API public site

# Set up request to the ip-api.com API

## We are only doing 4 for demonstration purposes because more might incur costs.
##
## If we were going to fix all the missing values in web_stats_df we would select all of the web stat rows with missing data 
## with something like this: 
##
##    web_stats_null_rows_df = web_stats_df[web_stats_df['CITY'].isnull() | web_stats_df['REGION'].isnull() | web_stats_df['REGIONCODE'].isnull() | web_stats_df['COUNTRY'].isnull()]
##
## and send all of the IP values in the API request instead of only four. 

In [12]:
import time
import requests
import json
from pandas import json_normalize

url = "http://ip-api.com/batch?fields=status,message,country,region,regionName,city,zip,query"

#Build payload for request. 
payload = "[\"154.54.249.207\", \"91.198.174.192\", \"141.98.81.23\", \"66.249.75.38\"]"

#Create content-type header for request. We're sending the request payload as plain text
headers = {
  'Content-Type': 'text/plain'
}

In [13]:
#Send the header and payload using a POST request, show the response status code (200 = OK) 
# and, since the result should be small, print the raw json response payload so we can see what we're dealing with

In [14]:
response = requests.request("POST", url, headers=headers, data=payload)
print('http status code = ' + str(response.status_code))
print('response json payload = ' + response.text)

http status code = 200
response json payload = [{"status":"success","country":"France","region":"IDF","regionName":"Île-de-France","city":"Nozay","zip":"91620","query":"154.54.249.207"},{"status":"success","country":"Netherlands","region":"NH","regionName":"North Holland","city":"Amsterdam","zip":"1012","query":"91.198.174.192"},{"status":"success","country":"Netherlands","region":"DR","regionName":"Drenthe","city":"Meppel","zip":"7941","query":"141.98.81.23"},{"status":"success","country":"United States","region":"WA","regionName":"Washington","city":"Seattle","zip":"98103","query":"66.249.75.38"}]


In [15]:
# Parse the json response payload text into a pandas dictionary, create a pandas dataframe from the dictionary and print it

In [16]:
locations = json.loads(response.text)

location_df = pd.DataFrame(locations)

print(location_df)

    status        country region     regionName       city    zip  \
0  success         France    IDF  Île-de-France      Nozay  91620   
1  success    Netherlands     NH  North Holland  Amsterdam   1012   
2  success    Netherlands     DR        Drenthe     Meppel   7941   
3  success  United States     WA     Washington    Seattle  98103   

            query  
0  154.54.249.207  
1  91.198.174.192  
2    141.98.81.23  
3    66.249.75.38  


In [17]:
# rename some of the location_df columns to the same name of the web_stats_df columns for easier handling
location_df.rename(columns={"country": "COUNTRY", "region_code": "REGIONCODE", "regionName": "REGION", "city": "CITY", "query": "IP"}, inplace = True)

print(location_df)

    status        COUNTRY region         REGION       CITY    zip  \
0  success         France    IDF  Île-de-France      Nozay  91620   
1  success    Netherlands     NH  North Holland  Amsterdam   1012   
2  success    Netherlands     DR        Drenthe     Meppel   7941   
3  success  United States     WA     Washington    Seattle  98103   

               IP  
0  154.54.249.207  
1  91.198.174.192  
2    141.98.81.23  
3    66.249.75.38  


In [18]:
# loop thru the locations data frame index and locate each web_stats dataframe
# IP addresses with each location IP at the current index. If found replace all the web stats dataframe location values 
# with the locations data frame location values
#
# Calculate and print the missing counts.These counts can be compared to the counts for the original file above

In [19]:
# There may be a better way to do this but we'll stick with this for now
for ind in location_df.index:
    web_stats_df.loc[web_stats_df['IP'] == location_df['IP'][ind], 'COUNTRY'] = location_df['COUNTRY'][ind]
    web_stats_df.loc[web_stats_df['IP'] == location_df['IP'][ind], 'REGION'] = location_df['REGION'][ind]
    web_stats_df.loc[web_stats_df['IP'] == location_df['IP'][ind], 'CITY'] = location_df['CITY'][ind]
    web_stats_df.loc[web_stats_df['IP'] == location_df['IP'][ind], 'ZIP'] = location_df['zip'][ind]

missing_values_count = web_stats_df.isnull().sum()
missing_values_count[0:24]

RECORDID           0
EPOCH              0
DATE               0
DATESTAMP          0
SESSION            0
IP                 0
HOSTNAME           0
PAGE               0
L1             96272
L2            243227
ARTID              0
EDITKEY       301722
FIRSTNAME     308145
LASTNAME      301786
CITY          197882
REGION        166694
REGIONCODE    172491
COUNTRY         1600
REF                0
IPOLD         333351
BROWSER       333351
MEMID              0
TYPE           96249
PUBLISH            0
dtype: int64